In [1]:
import os
os.getcwd()
os.chdir("../")
os.getcwd()

'd:\\workspace\\git_projects\\NBFI-Loan-Repayment'

In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import time
import re
from datetime import datetime
from utils.dbmanager import create_sql_connection, ingest_data_to_sql, query_data
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
from scipy import stats
import random
random.seed(42)

['SQL Server', 'ODBC Driver 17 for SQL Server', 'SQL Server Native Client RDA 11.0', 'Microsoft Access Driver (*.mdb, *.accdb)', 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)', 'Microsoft Access Text Driver (*.txt, *.csv)', 'Microsoft Access dBASE Driver (*.dbf, *.ndx, *.mdx)']


In [3]:
server = 'nbfi-loan-def-prediction.database.windows.net'     
database = 'NBFIDB'  
username = 'adminuser'       
password = 'Rambo@12345' 

# Establish connection
conn = create_sql_connection(server, database, username, password)

Connection established successfully!


In [ ]:
data = query_data(conn, "SELECT * FROM Silver.CustomerRaw")
data.head(5)

In [4]:
data = pd.read_csv('data/interim/NBFIDataSilver.csv')
data.shape

(121787, 40)

In [5]:
data.head()

,ID,Client_Income,Car_Owned,Bike_Owned,Active_Loan,House_Own,Child_Count,Credit_Amount,Loan_Annuity,Accompany_Client,Client_Income_Type,Client_Education,Client_Marital_Status,Client_Gender,Loan_Contract_Type,Client_Housing_Type,Population_Region_Relative,Age_Days,Employed_Days,Registration_Days,ID_Days,Own_House_Age,Mobile_Tag,Homephone_Tag,Workphone_Working,Client_Occupation,Client_Family_Members,Client_City_Rating,Application_Process_Day,Application_Process_Hour,Client_Permanent_Match_Tag,Client_Contact_Work_Tag,Type_Organization,Score_Source_1,Score_Source_2,Score_Source_3,Social_Circle_Default,Phone_Change,Credit_Bureau,Defaulters
0,12142509,6750.0,0.0,0.0,1.0,0.0,0.0,61190.55,3416.85,Alone,Commercial,Secondary,M,Male,CL,Home,0.028663,13957.0,1062.0,6123.0,383.0,NaN,1,1,0,Sales,2.0,2.0,6.0,17,Yes,Yes,Self-employed,0.568066,0.478787,0.511190,0.0186,63.0,1.0,0
1,12138936,20250.0,1.0,0.0,1.0,1.0,0.0,15282.00,1826.55,Alone,Service,Graduation,M,Male,CL,Home,0.008575,14162.0,4129.0,7833.0,21.0,0.0,1,0,1,NaN,2.0,2.0,3.0,10,Yes,Yes,Government,0.563360,0.215068,0.511190,NaN,755.0,1.0,0
2,12181264,18000.0,0.0,0.0,1.0,0.0,1.0,59527.35,2788.20,Alone,Service,Graduation dropout,W,Male,CL,Family,0.022800,16790.0,5102.0,4493.0,331.0,NaN,1,0,0,Realty agents,2.0,2.0,4.0,12,Yes,Yes,Self-employed,NaN,0.552795,0.329655,0.0742,277.0,0.0,0
3,12188929,15750.0,0.0,0.0,1.0,1.0,0.0,53870.40,2295.45,Alone,Retired,Secondary,M,Male,CL,Home,0.010556,23195.0,365243.0,4493.0,775.0,NaN,1,0,0,NaN,2.0,3.0,2.0,15,Yes,Yes,XNA,NaN,0.135182,0.631355,NaN,1700.0,3.0,0
4,12133385,33750.0,1.0,0.0,1.0,0.0,2.0,133988.40,3547.35,Alone,Commercial,Secondary,M,Female,CL,Home,0.020713,11366.0,2977.0,5516.0,4043.0,6.0,1,0,0,Laborers,4.0,1.0,3.0,12,Yes,Yes,Business Entity Type 3,0.508199,0.301182,0.355639,0.2021,674.0,1.0,0


### Categorical Columns Processing

In [6]:
retain_categories = ['Alone', 'Relative', 'Partner']

# Replace all other categories with 'Other'
data["Accompany_Client"] = data["Accompany_Client"].apply(lambda x: x if x in retain_categories else 'Other')
data['Accompany_Client'].value_counts()

Accompany_Client
Alone       99098
Relative    15739
Partner      4514
Other        2436
Name: count, dtype: int64

In [7]:
retain_categories = ['Service', 'Commercial', 'Retired', 'Govt Job']
    
# Replace all other categories with 'Other'
data["Client_Income_Type"] = data["Client_Income_Type"].apply(lambda x: x if x in retain_categories else 'Other')
data["Client_Income_Type"].value_counts()

Client_Income_Type
Service       64693
Commercial    27748
Retired       21030
Govt Job       8299
Other            17
Name: count, dtype: int64

In [8]:
data["Client_Education"] = data["Client_Education"].replace({'Post Grad': 'Graduation'})
data["Client_Education"].value_counts()

Client_Education
Secondary             87511
Graduation            28864
Graduation dropout     3957
Junior secondary       1455
Name: count, dtype: int64

In [9]:
data["Client_Gender"] = data["Client_Gender"].replace({'XNA': 'Male'})
data["Client_Gender"].value_counts()

Client_Gender
Male      80836
Female    40951
Name: count, dtype: int64

In [11]:
pd.crosstab(data['Type_Organization'], data['Defaulters'])
del data['Type_Organization'] # as it has lot of categories

In [13]:
data['Defaulters'] = data['Defaulters'].replace({'N': 0, 'Y': 1})
data['Defaulters'].value_counts()

Defaulters
0    111949
1      9838
Name: count, dtype: int64

In [14]:
data.shape

(121787, 39)

In [16]:
data.head()

,ID,Client_Income,Car_Owned,Bike_Owned,Active_Loan,House_Own,Child_Count,Credit_Amount,Loan_Annuity,Accompany_Client,Client_Income_Type,Client_Education,Client_Marital_Status,Client_Gender,Loan_Contract_Type,Client_Housing_Type,Population_Region_Relative,Age_Days,Employed_Days,Registration_Days,ID_Days,Own_House_Age,Mobile_Tag,Homephone_Tag,Workphone_Working,Client_Occupation,Client_Family_Members,Client_City_Rating,Application_Process_Day,Application_Process_Hour,Client_Permanent_Match_Tag,Client_Contact_Work_Tag,Score_Source_1,Score_Source_2,Score_Source_3,Social_Circle_Default,Phone_Change,Credit_Bureau,Defaulters
0,12142509,6750.0,0.0,0.0,1.0,0.0,0.0,61190.55,3416.85,Alone,Commercial,Secondary,M,Male,CL,Home,0.028663,13957.0,1062.0,6123.0,383.0,NaN,1,1,0,Sales,2.0,2.0,6.0,17,Yes,Yes,0.568066,0.478787,0.511190,0.0186,63.0,1.0,0
1,12138936,20250.0,1.0,0.0,1.0,1.0,0.0,15282.00,1826.55,Alone,Service,Graduation,M,Male,CL,Home,0.008575,14162.0,4129.0,7833.0,21.0,0.0,1,0,1,NaN,2.0,2.0,3.0,10,Yes,Yes,0.563360,0.215068,0.511190,NaN,755.0,1.0,0
2,12181264,18000.0,0.0,0.0,1.0,0.0,1.0,59527.35,2788.20,Alone,Service,Graduation dropout,W,Male,CL,Family,0.022800,16790.0,5102.0,4493.0,331.0,NaN,1,0,0,Realty agents,2.0,2.0,4.0,12,Yes,Yes,NaN,0.552795,0.329655,0.0742,277.0,0.0,0
3,12188929,15750.0,0.0,0.0,1.0,1.0,0.0,53870.40,2295.45,Alone,Retired,Secondary,M,Male,CL,Home,0.010556,23195.0,365243.0,4493.0,775.0,NaN,1,0,0,NaN,2.0,3.0,2.0,15,Yes,Yes,NaN,0.135182,0.631355,NaN,1700.0,3.0,0
4,12133385,33750.0,1.0,0.0,1.0,0.0,2.0,133988.40,3547.35,Alone,Commercial,Secondary,M,Female,CL,Home,0.020713,11366.0,2977.0,5516.0,4043.0,6.0,1,0,0,Laborers,4.0,1.0,3.0,12,Yes,Yes,0.508199,0.301182,0.355639,0.2021,674.0,1.0,0


In [18]:
# data.isna().sum()

### Feature Engineering

1. Financial Ratios

In [19]:
data["Debt_Income_Ratio"] = data["Credit_Amount"] / data["Client_Income"]
data["Annuity_Income_Ratio"] = data["Loan_Annuity"] / data["Client_Income"]
data["Loan_Duration"] = data["Credit_Amount"] / data["Loan_Annuity"]


2. Employment & Stability Features

In [20]:
data["Age_Years"] = data["Age_Days"] // 365
data["Employed_Years"] = data["Employed_Days"].apply(lambda x: x // 365 if x > 0 else 0)
data["Registration_Years"] = data["Registration_Days"] // 365
data["ID_Years"] = data["ID_Days"] // 365


In [23]:
data.describe()

,ID,Client_Income,Car_Owned,Bike_Owned,Active_Loan,House_Own,Child_Count,Credit_Amount,Loan_Annuity,Population_Region_Relative,Age_Days,Employed_Days,Registration_Days,ID_Days,Own_House_Age,Mobile_Tag,Homephone_Tag,Workphone_Working,Client_Family_Members,Client_City_Rating,Application_Process_Day,Application_Process_Hour,Score_Source_1,Score_Source_2,Score_Source_3,Social_Circle_Default,Phone_Change,Credit_Bureau,Defaulters,Debt_Income_Ratio,Annuity_Income_Ratio,Loan_Duration,Age_Years,Employed_Years,Registration_Years,ID_Years
count,1.217870e+05,1.217870e+05,121787.000000,121787.000000,121787.000000,121787.000000,121787.000000,121787.000000,121787.000000,121787.000000,121787.000000,121787.000000,121787.000000,121787.000000,41737.000000,121787.000000,121787.000000,121787.000000,121787.000000,121787.000000,121787.000000,121787.000000,52996.000000,121787.000000,121787.000000,59899.000000,121787.000000,121787.000000,121787.000000,121787.000000,121787.000000,121787.000000,121787.000000,121787.000000,121787.000000,121787.000000
mean,1.216093e+07,1.686508e+04,0.332770,0.322407,0.484305,0.701323,0.405355,60044.941824,2721.245116,0.022602,16018.664472,65206.029699,4960.991526,2999.869222,12.154252,0.999992,0.200506,0.281220,2.151354,2.029478,3.156412,12.061181,0.501209,0.518631,0.511190,0.117424,955.853851,1.755581,0.080780,3.971628,0.181911,21.837388,43.396438,178.134842,13.104576,7.710125
std,3.517759e+04,1.136697e+04,0.471207,0.467400,0.499756,0.457680,0.721413,39744.889817,1432.383543,0.405385,4301.635740,137319.239708,3463.166284,1475.444259,12.053055,0.002865,0.400380,0.449597,0.903917,0.499455,1.741600,3.231052,0.211204,0.722962,0.171514,0.107978,816.114541,1.744073,0.272499,2.683586,0.095317,8.775597,11.781950,376.146776,9.483085,4.039253
min,1.210000e+07,2.565000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,4500.000000,217.350000,0.000533,7676.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.014568,0.000005,0.000527,0.000000,0.000000,0.000000,0.000000,0.037500,0.003850,1.653655,21.000000,0.000000,0.000000,0.000000
25%,1.213046e+07,1.125000e+04,0.000000,0.000000,0.000000,0.000000,0.000000,27450.000000,1687.500000,0.010032,12511.000000,961.000000,2101.000000,1789.000000,5.000000,1.000000,0.000000,0.000000,2.000000,2.000000,2.000000,10.000000,0.333481,0.402068,0.424130,0.057700,287.000000,1.000000,0.000000,2.040667,0.115649,15.540384,34.000000,2.000000,5.000000,4.000000
50%,1.216093e+07,1.575000e+04,0.000000,0.000000,0.000000,1.000000,0.000000,52359.750000,2561.400000,0.019101,15734.000000,2212.000000,4493.000000,3242.000000,9.000000,1.000000,0.000000,0.000000,2.000000,2.000000,3.000000,12.000000,0.504723,0.553179,0.511190,0.088700,755.000000,1.000000,0.000000,3.312800,0.163925,20.000000,43.000000,6.000000,12.000000,8.000000
75%,1.219139e+07,2.025000e+04,1.000000,1.000000,1.000000,1.000000,1.000000,80865.000000,3408.750000,0.026392,19544.000000,5386.500000,7350.000000,4263.000000,15.000000,1.000000,0.000000,1.000000,3.000000,2.000000,5.000000,14.000000,0.673890,0.659828,0.631355,0.148500,1550.000000,3.000000,0.000000,5.191578,0.230207,27.630389,53.000000,14.000000,20.000000,11.000000
max,1.222186e+07,1.800009e+06,1.000000,1.000000,1.000000,1.000000,19.000000,405000.000000,22500.000000,100.000000,25201.000000,365243.000000,23738.000000,7197.000000,69.000000,1.000000,1.000000,1.000000,16.000000,3.000000,6.000000,23.000000,0.945741,100.000000,0.896010,1.000000,4185.000000,22.000000,1.000000,49.227200,1.570600,167.419328,69.000000,1000.000000,65.000000,19.000000


3. Interaction Features

In [24]:
data["Employment_to_Age_Ratio"] = data["Employed_Years"] / data["Age_Years"]
data["Credit_to_Age_Ratio"] = data["Credit_Amount"] / data["Age_Years"]
data["Family_Size"] = data["Client_Family_Members"] + data["Child_Count"]

In [25]:
data.head()

,ID,Client_Income,Car_Owned,Bike_Owned,Active_Loan,House_Own,Child_Count,Credit_Amount,Loan_Annuity,Accompany_Client,Client_Income_Type,Client_Education,Client_Marital_Status,Client_Gender,Loan_Contract_Type,Client_Housing_Type,Population_Region_Relative,Age_Days,Employed_Days,Registration_Days,ID_Days,Own_House_Age,Mobile_Tag,Homephone_Tag,Workphone_Working,Client_Occupation,Client_Family_Members,Client_City_Rating,Application_Process_Day,Application_Process_Hour,Client_Permanent_Match_Tag,Client_Contact_Work_Tag,Score_Source_1,Score_Source_2,Score_Source_3,Social_Circle_Default,Phone_Change,Credit_Bureau,Defaulters,Debt_Income_Ratio,Annuity_Income_Ratio,Loan_Duration,Age_Years,Employed_Years,Registration_Years,ID_Years,Employment_to_Age_Ratio,Credit_to_Age_Ratio,Family_Size
0,12142509,6750.0,0.0,0.0,1.0,0.0,0.0,61190.55,3416.85,Alone,Commercial,Secondary,M,Male,CL,Home,0.028663,13957.0,1062.0,6123.0,383.0,NaN,1,1,0,Sales,2.0,2.0,6.0,17,Yes,Yes,0.568066,0.478787,0.511190,0.0186,63.0,1.0,0,9.065267,0.506200,17.908468,38.0,2.0,16.0,1.0,0.052632,1610.277632,2.0
1,12138936,20250.0,1.0,0.0,1.0,1.0,0.0,15282.00,1826.55,Alone,Service,Graduation,M,Male,CL,Home,0.008575,14162.0,4129.0,7833.0,21.0,0.0,1,0,1,NaN,2.0,2.0,3.0,10,Yes,Yes,0.563360,0.215068,0.511190,NaN,755.0,1.0,0,0.754667,0.090200,8.366593,38.0,11.0,21.0,0.0,0.289474,402.157895,2.0
2,12181264,18000.0,0.0,0.0,1.0,0.0,1.0,59527.35,2788.20,Alone,Service,Graduation dropout,W,Male,CL,Family,0.022800,16790.0,5102.0,4493.0,331.0,NaN,1,0,0,Realty agents,2.0,2.0,4.0,12,Yes,Yes,NaN,0.552795,0.329655,0.0742,277.0,0.0,0,3.307075,0.154900,21.349742,46.0,13.0,12.0,0.0,0.282609,1294.072826,3.0
3,12188929,15750.0,0.0,0.0,1.0,1.0,0.0,53870.40,2295.45,Alone,Retired,Secondary,M,Male,CL,Home,0.010556,23195.0,365243.0,4493.0,775.0,NaN,1,0,0,NaN,2.0,3.0,2.0,15,Yes,Yes,NaN,0.135182,0.631355,NaN,1700.0,3.0,0,3.420343,0.145743,23.468340,63.0,1000.0,12.0,2.0,15.873016,855.085714,2.0
4,12133385,33750.0,1.0,0.0,1.0,0.0,2.0,133988.40,3547.35,Alone,Commercial,Secondary,M,Female,CL,Home,0.020713,11366.0,2977.0,5516.0,4043.0,6.0,1,0,0,Laborers,4.0,1.0,3.0,12,Yes,Yes,0.508199,0.301182,0.355639,0.2021,674.0,1.0,0,3.970027,0.105107,37.771407,31.0,8.0,15.0,11.0,0.258065,4322.206452,6.0


In [26]:
data.shape

(121787, 49)

In [ ]:
# Ingest Data into Bronze Table
if conn:
    ingest_data_to_sql(conn, data.iloc[:1001,:], table_name="CustomerRaw", schema="Gold", batch_size=1000)
    #conn.close()  # Close connection after insertion

In [27]:
data.to_csv('data/interim/NBFIDataGold.csv', index=False)